
# Biogéophysique#

*Notebook créé par Mathieu Poupon*

La géophysique est une discipline scientifique dont l'objectif est l'étude des processus physiques sur déroulant sur Terre, telles la dynamique des plaques tectoniques, la circulation atmosphérique ou encore les variations du champ magnétique.

**Plan:** <a id='Plan'></a>
* [1 - Introduction aux mesures géophysiques](#Introduction)
* [2 - Description et prise en main des données](#Description)
* [3 - Visualisation des données](#Visualisation)
* [4 - Exploitation des données](#Exploitation)
* [5 - Aller plus loin](#Extra)

# 1. Introduction aux mesures géophysiques <a id='Introduction'></a>
[Retour au menu](#Plan)

Pour comprendre ces processus, les scientifiques ont besoin de données. De nombreux instruments de mesures ont été développés pour récolter ces données. On distingue deux types de mesures :
* Directes : Le résultat est obtenu directement à partir d'un instrument de mesure (ex: un thermomètre indique la température d'un plat)
* Indirectes : Le résultat est obtenu par un calcul à partir des données mesurées par l'instrument (ex: un radar indique la vitesse d'une voiture après un calcul utilisant le temps de parcours d'ondes)

Dans ce notebook, l'objectif est d'étudier et de comprendre la distribution des reliefs observables à la surface de la Terre. Pour cela, des mesures de topographie de surface et de bathymétrie sont nécessaires.

## Mesurer les reliefs

Il existe plusieurs méthodes pour mesurer les reliefs sur Terre. Les mesures GPS sont utilisées pour connaître des altitudes ponctuelles sur les continents, tandis que la bathymétrie permet de mesurer le relief du plancher océanique.

### Mesures GPS

Pour déterminer l'altitude d'un point, quatre satellites sont nécessaires. Une constellation de satellites émet des ondes dont le signal est capté par des récepteurs GPS. En mesurant le temps de parcours des ondes, on peut remonter à la distance entre le récepteur et le satellite, puis à la position du récepteur par triangulation.

<img src="img/01_GNSS-GPS-GLONASS-GALILEO-3D-trilateration.jpg" width='600'>

Cette position (altitude géodésique) est calculée par le récepteur par rapport à un modèle définissant le niveau moyen des océans s'ils recouvraient l'ensemble du globe (ellipsoïde de référence). Pour obtenir l'altitude (orthométrique), il faut soustraire à cette mesure les variations de géoïde, qui correspond à une surface équipotentielle du champ de pesanteur ($\vec{g}$ est constante et normale à cette surface).

Ainsi:
$$
H_{orthométrique} = h_{géodésique} - N_{géoïde}
$$

<img src="img/01_three_surfaces_f.jpg" width='600'>


### Mesures bathymétriques

Pour mesurer le relief du plancher océanique des sondeurs bathymétriques sont utilisés. Ils sont placés sur des bateaux qui réalisent des sections dans l'océan. Ces sondeurs génèrent un signal acoustique se réfléchissant sur le plancher. La profondeur est déduite du temps de trajet de l'onde réfléchie en direction du bateau.

<img src="img/01_Sondeur-multifaisceaux.jpg" width='400'>

# 2. Description et prise en main des données <a id='Description'></a>
[Retour au menu](#Plan)

Une fois réalisées, les mesures sont stockées dans des fichiers de format variables pour pouvoir être utilisées et en extraire de l'information. Les données utilisées dans ce notebook sont au format NETCDF (fichier se terminant par l'extension .nc). Il est possible de les ouvrir et de les manipuler avec plusieurs langages de programmation dont Python.

## 2.a. Importation des bibliothèques

Les bibliothèques regroupent un ensemble de fonction permettant de manipuler et de visualiser facilement des données.\
Pour pouvoir utiliser ces fonctions, il faut importer les bibliothèques dans notre code.

In [ ]:
import xarray as xr # Ouverture des fichiers .nc
import numpy as np # Manipulation rapide de structures matricielles
import matplotlib.pyplot as plt # Représentation graphique des données
import metpy # Manipulation de données géophysiques
from metpy.interpolate import cross_section # Sections dans des données 2 ou 3D
import cartopy.crs as ccrs # Représentation des données sur un planisphère
import cartopy.feature as cfeature # Ajout de paramètres graphiques (Continents, Rivières, ...)
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
warnings.filterwarnings("ignore")

La documentation de ces bibliothèques, décrivant leurs fonctionnalités, est accessible facilement en entrant leur nom dans un moteur de recherche.

## 2.b. Structure et ouverture des données

**Structure: Exemple des données de topographie**

Comment sont structurées les données ?
* Un **dataset** est un fichier regroupant un ensemble de **variables** mesurées selon une ou plusieurs **dimensions**, ainsi qu'un ensemble d'informations supplémentaire nommé **métadonnées**.  
* Les données topographiques sont stockées sous forme d'un **tableau à 2 dimensions** (analogue à une matrice).
* Le dataset **"topo"** contient donc **z** qui est la **variable** correspondant la mesure de la topographie (topo.z), ainsi que les **deux dimensions** latitude (topo.lat) et longitude (topo.lon) selon lesquelles sont définit les **coordonnées** de la grille sur laquelle est mesurée la variable z. 
* Il contient également des **métadonnées** telles que le titre du jeu de données, le nom des conventions utilisées, etc ...

**Ouverture et extraction des données de topographie**

In [ ]:
# Ouverture des données
path = "Data/ETOPO2v2g_f4.nc"
topo = xr.open_dataset(path)
topo = topo.rename_dims({'x':'lon','y':'lat'})
topo = topo.rename_vars({'x':'lon','y':'lat'})
topo = topo.metpy.parse_cf().squeeze()
topo

Pour utiliser les données, il suffit de taper le nom du **dataset** suivi du nom de la **variable**, par exemple:

In [ ]:
topo.z

# 3. Visualisation des données <a id='Visualisation'></a>

[Retour au menu](#Plan)

## 3.a. Représentation de l'ensemble des données

**Topographie:** \
Les données topographiques peuvent être représentées sur une carte 2D

In [ ]:
fig = plt.figure(figsize=(20,7),dpi=200)
ax = fig.add_subplot(111,projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)
ax.set_xticks(np.linspace(-180,180,9), crs=ccrs.PlateCarree())
ax.set_yticks(np.linspace(-90,90,7), crs=ccrs.PlateCarree())
ax.set_xlabel("Longitude [°E]")
ax.set_ylabel("Latitude [°N]")

c=ax.pcolormesh(topo.lon,topo.lat,topo.z,vmin=-6000,vmax=6000,cmap=plt.cm.jet)
plt.colorbar(c,label='Topographie [m]')
plt.show()

**Question:** Quelles structures géologiques sont observables sur cette figure ?

**Réponse:**

## 3.b Fonctions

Pour manipuler plus rapidement les données, on peut créer et utiliser des fonctions:
* zoom_topo: faire un zoom de la carte topographique
* coupe_topo: faire une coupe de la topographie entre deux points 
* zoom_coupe_topo: zoom_topo et coupe_topo en même temps

### 3.b.i. zoom_topo: faire un zoom de la carte topographique

La fonction **zoom_topo** permet d'effectuer un zoom de la carte topographique. \
Elle prend en argument les longitudes et latitudes bornant le zoom:
* La longitude (*°E*) doit être un nombre réel compris entre -180 et 180.
* La latitude (*°N*) doit être un nombre réel compris entier entre -90 et 90.

In [ ]:
def zoom_topo(lon_min, lon_max, lat_min, lat_max):
    mask_lon = np.logical_and(topo.lon>lon_min,topo.lon<lon_max)
    mask_lat = np.logical_and(topo.lat>lat_min,topo.lat<lat_max)
    mask = np.logical_and(mask_lon,mask_lat)

    fig = plt.figure(figsize=(20,7),dpi=200)
    ax = fig.add_subplot(111,projection=ccrs.PlateCarree())
    ax.set_xlim(lon_min,lon_max),ax.set_ylim(lat_min,lat_max)
    ax.set_xticks(np.linspace(lon_min,lon_max,9), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(lat_min,lat_max,7), crs=ccrs.PlateCarree())
    ax.set_xlabel("Longitude [°E]"),ax.set_ylabel("Latitude [°N]")
    
    c=ax.pcolormesh(topo.lon[mask_lon],topo.lat[mask_lat],topo.z[mask_lat,mask_lon],vmin=-6000,vmax=6000,cmap=plt.cm.jet)
    plt.colorbar(c,label='Topographie [m]')

Par exemple, pour faire un zoom entre -70°E et 10°E de longitude et 0°N et 60°N de latitude:

In [ ]:
zoom_topo(lon_min = -70, lon_max = 10, lat_min = 0, lat_max = 60)

### 3.b.ii. coupe_topo: faire une coupe de la topographie entre deux points
La fonction **coupe_topo** permet d'effectuer une coupe de la topographie entre deux points a et b. \
Elle prend en argument les longitudes et latitudes des points aux extrémités de la coupe:
* Les longitudes (*°E*) doivent être des nombres réels compris entre -180 et 180.
* Les latitudes (*°N*) doivent être des nombres réels compris entre -90 et 90.

In [ ]:
def coupe_topo(lon_a, lon_b, lat_a, lat_b):
    steps=1000
    start,end = (lat_a, lon_a),(lat_b, lon_b)
    cross = cross_section(topo, start, end,steps=steps)
    L = np.sqrt(((lat_b-lat_a)* 111.11)**2+((lon_b-lon_a)*111.11*np.cos((lat_a+lat_b)/2))**2)/steps
    
    fig = plt.figure(figsize=(15,5),dpi=200)
    ax = fig.add_subplot(111)
    ax.plot(cross.index*L,cross.z)
    ax.scatter(cross.index[[0,-1]]*L,cross.z[[0,-1]],c=['r','b'])
    ax.set_xlabel('Distance [km]'),ax.set_ylabel('Topographie [m]')

Par exemple, pour faire une coupe entre les points a et b de coordonées (-60°E;30°N) et (-30°E;20°N):

In [ ]:
coupe_topo(lon_a = -60, lon_b = -30, lat_a = 30, lat_b = 20)

### 3.b.iii. zoom_coupe_topo: Faire un zoom et une coupe de la topographie

La fonction **zoom_coupe_topo** permet de réprésenter à la fois un zoom et une coupe de la topographie. \
Elle prend en argument les longitudes et latitudes des points aux extrémités de la coupe:
* Les longitudes (*°E*) doivent être des nombres réels compris entre -180 et 180.
* Les latitudes (*°N*) doivent être des nombres réels compris entre -90 et 90.

In [ ]:
def zoom_coupe_topo(lon_a, lon_b, lat_a, lat_b):
    steps=1000
    start,end = (lat_a, lon_a),(lat_b, lon_b)
    cross = cross_section(topo, start, end,steps=steps)
    L = np.sqrt(((lat_b-lat_a)* 111.11)**2+((lon_b-lon_a)*111.11*np.cos((lat_a+lat_b)/2))**2)/steps
    
    lon_min, lon_max = np.min([lon_a, lon_b])-5,np.max([lon_a, lon_b])+5
    lat_min, lat_max = np.min([lat_a, lat_b])-5,np.max([lat_a, lat_b])+5
    
    mask_lon = np.logical_and(topo.lon>lon_min,topo.lon<lon_max)
    mask_lat = np.logical_and(topo.lat>lat_min,topo.lat<lat_max)
    mask = np.logical_and(mask_lon,mask_lat)

    fig = plt.figure(figsize=(15,5),dpi=200)
    ax = fig.add_subplot(122)
    ax.set_xlim(np.min([lon_a,lon_b])-5,np.max([lon_a,lon_b])+5)
    ax.set_ylim(np.min([lat_a,lat_b])-5,np.max([lat_a,lat_b])+5)
    ax.set_xlabel("Longitude [°E]"),ax.set_ylabel("Latitude [°N]")
    c=ax.pcolormesh(topo.lon[mask_lon],topo.lat[mask_lat],topo.z[mask_lat,mask_lon],vmin=-6000,vmax=6000,cmap=plt.cm.jet)
    plt.colorbar(c,label='Topographie [m]')
    
    ax.plot([lon_a,lon_b],[lat_a,lat_b],c='k')
    ax.scatter([lon_a,lon_b],[lat_a,lat_b],c=['r','g'])
    
    ax = fig.add_subplot(121)
    ax.plot(cross.index*L,cross.z)
    ax.scatter(cross.index[[0,-1]]*L,cross.z[[0,-1]],c=['r','g'])
    ax.set_xlabel('Distance [km]'),ax.set_ylabel('Topographie [m]')

Par exemple, pour faire une coupe entre les points a et b de coordonées (130°E;60°N) et (150°E;30°N):

In [ ]:
zoom_coupe_topo(lon_a = 130, lon_b = 150, lat_a = 60, lat_b = 30)

# 4. Exploitation des données <a id='Exploitation'></a>

[Retour au menu](#Plan)

En utilisant les données et les fonctions précédement présentées ci-dessus, répondrez aux questions suivantes:

### 4.a. Dorsales rapide vs dorsales lente

* Calculer la pente de la dorsale Atlantique:

In [ ]:
zoom_coupe_topo(lon_a = -30, lon_b = -60, lat_a = 27.5, lat_b = 30)

* Calculer la pente de la dorsale Pacifique

In [ ]:
zoom_coupe_topo(lon_a = -135, lon_b = -90, lat_a = -45, lat_b = -50)

**Question:** Les valeurs sont-elles identiques ? Comment expliquer cela ?

**Réponse:**

### 4.b. Alignement de points chauds
* Identifier une série de point chaud dans l'océan Indien

In [ ]:
zoom_topo(lon_min = 60, lon_max = 120, lat_min = -40, lat_max = 10)

### 4.c. Distribution des reliefs

Une courbe hypsométrique est une fonction de répartition des élévations d'une zone géographique. Elle permet d'étudier la distribution des reliefs de cette zone.

In [ ]:
def hypsometric(pas):
    z = np.max(topo.z)
    Prop,Prop_cum,Z = [0], [0], [z+pas]

    while z>np.min(topo.z): 
        mask = np.logical_and(topo.z > (z - pas),topo.z < z)
        prop = np.sum(mask)/np.size(mask)
        prop_cum = prop + Prop_cum[-1]
        Prop.append(prop),Z.append(z-pas/2),Prop_cum.append(prop_cum)
        z = z - pas
        
    return np.array(Prop),np.array(Prop_cum),np.array(Z)

In [ ]:
Prop,Prop_cum,Z = hypsometric(pas = 100)

fig = plt.figure(figsize=(15,5), dpi = 200)
fig.suptitle('Courbe hypsométrique', fontsize=14, fontweight = 'bold')
ax = fig.add_subplot(121)
ax.plot(Prop,Z)
ax.set_xlim(0.05,-0.001),ax.set_ylabel('Topographie [m]'),ax.set_xlabel('Fraction [%]')
ax = fig.add_subplot(122)
ax.plot(Prop_cum,Z),ax.set_xlabel('Fraction cumulée [%]')
plt.show()

# 5. Aller plus loin <a id='Extra'></a>

[Retour au menu](#Plan)

Avec les quelques fonctions précédement définies, vous pouver étudier librement ces données

In [ ]:
"""
Espace de code
"""

In [ ]:
!conda env export --from-history -f environment.yml